# Item-to-Item

## Основные выгоды от использования этого подхода CF

1. <b> Стабильность </b> : Рейтинги пользователей не постоянный и зависят от множества факторов. <a href="https://dl.acm.org/doi/10.1561/1100000009">(Michael D. Ekstrand, <i>et al.</i> 2011)</a>. 
2. <b> Масштабирование </b> : В данном подходе можно использовать заранее рассчитанные матрицы элементов <a href="https://dl.acm.org/doi/10.1145/371920.372071">(Sarwar <i>et al.</i> 2001)</a>, <a href="https://dl.acm.org/doi/10.1561/1100000009">(Michael D. Ekstrand, <i>et al.</i> 2011)</a>.

## Algorithm : item-to-item

Подход item-based CF описан здесь <a href="http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.449.1171&rep=rep1&type=pdf">(B. Sarwar et al. 2001)</a><a href="https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.554.1671&rep=rep1&type=pdf">(George Karypis 2001)</a> 

### Import useful requirements

In [2]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split as sklearn_train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix

import pandas as pd
import numpy as np
import zipfile

import os
import sys

In [3]:
import warnings
warnings.filterwarnings('ignore')

### Загружаем наборы данных

In [5]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

### 1. Найдем схожесть между элементами

1. найдем пользователей, которые оценили один и то же элемент
2. нормализуем рейтинги
3. рассчитаем косинусное расстояние для нормализованных рейтингов

In [8]:
def normalize():
    # средний рейтинг для всех
    mean = ratings.groupby(by='userId', as_index=False)['rating'].mean()
    norm_ratings = pd.merge(ratings, mean, suffixes=('','_mean'), on='userId')
    
    # нормализуем все рейтинги относительно среднего
    norm_ratings['norm_rating'] = norm_ratings['rating'] - norm_ratings['rating_mean']
    return mean.to_numpy()[:, 1], norm_ratings

In [9]:
mean, norm_ratings = normalize()
np_ratings = norm_ratings.to_numpy()
norm_ratings.head()

,userId,movieId,rating,timestamp,rating_mean,norm_rating
0,1,31,2.5,1260759144,2.55,-0.05
1,1,1029,3.0,1260759179,2.55,0.45
2,1,1061,3.0,1260759182,2.55,0.45
3,1,1129,2.0,1260759185,2.55,-0.55
4,1,1172,4.0,1260759205,2.55,1.45


In [12]:
# создадим CSR матрицу
def item_representation(ratings):    
    return csr_matrix(
        pd.crosstab(ratings.movieId, ratings.userId, ratings.norm_rating, aggfunc=sum).fillna(0).values
    )

In [13]:
R = item_representation(norm_ratings)

#### используем такой же подход K-NN из sklearn, как и в User-based

In [14]:
def create_model(rating_matrix, k=21, metric="cosine"):
    """
    - создание модели с базовыми параметрами
    """
    model = NearestNeighbors(metric=metric, n_neighbors=k, algorithm='brute')
    
    model.fit(rating_matrix)    
    return model

#### Расчет схожести

In [15]:
def nearest_neighbors(rating_matrix, model):
    """
    :param rating_matrix : матрица рейтингов (nb_users, nb_items)
    :param model : модель knn  
    """    
    similarities, neighbors = model.kneighbors(rating_matrix)        
    return similarities[:, 1:], neighbors[:, 1:]

#### Ajusted Cosine Similarity (скорректированое)

Работа с элементами, лучше всего заменить обычное косинусное сходство на скорректированое, т.к. пользователи могут завышать или занижать оценки. Каждый ставит оценку по своему "ощущению"

\begin{equation}
 w_{i,j}= \frac{\sum_{u\in U}(r_{u,i}-\bar{r}_u)(r_{u,j}-\bar{r}_u)}{\sqrt{\sum_{u\in U} (r_{u,i}-\bar{r}_u)^2}\sqrt{\sum_{u\in U} (r_{u,j}-\bar{r}_u)^2}}.
\end{equation}

In [69]:
def save_similarities(similarities, neighbors, dataset_name):    
    base_dir = 'sims'
    save_dir = os.path.join(base_dir, dataset_name)
    os.makedirs(save_dir, exist_ok=True)    
    similarities_file_name = os.path.join(save_dir, 'similarities.npy')
    neighbors_file_name = os.path.join(save_dir, 'neighbors.npy')    
    try:
        np.save(similarities_file_name, similarities)
        np.save(neighbors_file_name, neighbors)        
    except ValueError as error:
        print(f"Ошибка во время сохранения : \n ValueError : {error}")

        
def load_similarities(dataset_name, k=20):
    base_dir = 'sims'
    save_dir = os.path.join(base_dir, dataset_name)    
    similiraties_file = os.path.join(save_dir, 'similarities.npy')
    neighbors_file = os.path.join(save_dir, 'neighbors.npy')    
    similarities = np.load(similiraties_file)
    neighbors = np.load(neighbors_file)    
    return similarities[:,:k], neighbors[:,:k]


def cosine(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))


def adjusted_cosine(np_ratings, nb_items, items, dataset_name):
    similarities = np.zeros(shape=(nb_items, nb_items))
    similarities.fill(-1)
    
    def _progress(count):
        sys.stdout.write('\rComputing similarities. Progress status : %.1f%%' % (float(count / nb_items)*100.0))
        sys.stdout.flush()
        
    for i in items[:-1]:
        for j in items[i+1:]:         
            if i <= 1000 and j <= 1000:
            
                scores = np_ratings[(np_ratings[:, 1] == i) | (np_ratings[:, 1] == j), :]
                vals, count = np.unique(scores[:,0], return_counts = True)
                scores = scores[np.isin(scores[:,0], vals[count > 1]),:]

                if scores.shape[0] > 2:
                    x = scores[scores[:, 1].astype('int') == i, 4]
                    y = scores[scores[:, 1].astype('int') == j, 4]
                    w = cosine(x, y)
                    similarities[i, j] = w
                    similarities[j, i] = w
            else:
                pass
            
        _progress(i)
    _progress(nb_items)
    
    # получаем и сортируем ближайших соседей
    neighbors = np.flip(np.argsort(similarities), axis=1)
    
    # сортируем рейтинги 
    similarities = np.flip(np.sort(similarities), axis=1)
    
    # сохраняем
    save_similarities(similarities, neighbors, dataset_name=dataset_name) 
    
    return similarities, neighbors

# очень много объектов, предложите идею сокращения размерности?

In [70]:
# очень много объектов, предложите идею сокращения размерности?

nb_items = 1000 #ratinvalue_countsId.nunique()
similarities, neighbors = adjusted_cosine(np_ratings, nb_items=nb_items, items = ratings.movieId.unique(), dataset_name='ml')

Computing similarities. Progress status : 100.0%%%

### 2. Top N рекомендаций для пользователя

#### Найдем элементы-кандидаты для рекомендации

1. Выбирем элементы, который пользователь уже оценил
2. Сделаем расчет сходства по всем элементам
3. Исключим уже отмеченные пользователем элементы


In [71]:
def candidate_items(userid):
    """
    :param userid : пользователь чей набор рекомендаций собираем 
    :return : элементы в наборе
    """
    
    # 1. Выбирем элементы, который пользователь уже оценил
    I_u = np_ratings[np_ratings[:, 0] == userid]
    I_u = I_u[:, 1].astype('int')
    
    # 2. Сделаем расчет сходства по всем элементам
    c = set()
        
    for iid in I_u:    
        # добавим соседей выбранного элемента
        c.update(neighbors[iid])
        
    c = list(c)
    # 3. Исключим уже отмеченные пользователем элементы
    candidates = np.setdiff1d(c, I_u, assume_unique=True)
    
    return I_u, candidates

In [77]:
i_u, u_candidates = candidate_items(2)

In [79]:
print('кол-во элементов по пользователю : ', len(i_u))
print('кол-во возможных рекомендаций для пользователя : ', len(u_candidates))

кол-во элементов по пользователю :  76
кол-во возможных рекомендаций для пользователя :  924


#### найдем сходство каждого элемента из набора

In [81]:
def similarity_with_Iu(c, I_u):
    """
    функция поиска сходства
    """
    w = 0    
    for iid in I_u :        
        # схожесть между элементами (по элементно)
        # наш совместный код
        
        
    return w

#### сортировка по рейтингу

In [82]:
def rank_candidates(candidates, I_u):
    """
    расчет рангов и общая сортировка всех элементов
    """
    
    # из списка элементов вычисляем для каждого схожесть
    sims = [similarity_with_Iu(c, I_u) for c in candidates]  
    mapping = list(zip(candidates, sims))
    
    # сортируем результаты по рейтингу
    ranked_candidates = sorted(mapping, key=lambda couple:couple[1], reverse=True)    
    return ranked_candidates

### Топ рекомендаций

In [104]:
def topn_recommendation(userid, N=10):
    """
    функция отбора Топ-N рекомендаций
    """
    # поиск элементов
    I_u, candidates = candidate_items(userid)
    
    #  сортировка элементов 
    ranked_candidates = rank_candidates(candidates, I_u)
    
    # отбор Топ-N
    topn = pd.DataFrame(ranked_candidates[:N], columns=['movieId','similarity_with_Iu'])    
    topn = pd.merge(topn, movies, on='movieId', how='inner')    
    return topn

In [105]:
topn_recommendation(2)

,movieId,similarity_with_Iu,title,genres
0,1,76.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,76.0,Jumanji (1995),Adventure|Children|Fantasy
2,3,76.0,Grumpier Old Men (1995),Comedy|Romance
3,5,76.0,Father of the Bride Part II (1995),Comedy
4,6,76.0,Heat (1995),Action|Crime|Thriller
5,7,76.0,Sabrina (1995),Comedy|Romance
6,11,76.0,"American President, The (1995)",Comedy|Drama|Romance
7,16,76.0,Casino (1995),Crime|Drama
8,19,76.0,Ace Ventura: When Nature Calls (1995),Comedy
9,21,76.0,Get Shorty (1995),Comedy|Crime|Thriller


## Top N c предиктом лучших рекомендаций

### Функция предикта

\begin{equation}
 \hat{r}_{u,i}=\frac{\sum_{j\in S^{(i)}}r_{u,j}\cdot w_{i,j}}{\sum_{j\in S^{(i)}}|w_{i,j}|}
\end{equation}

In [106]:
def predict(userid, itemid):
    """
    Создает предикт для пользователя относительно элемента
    """
    
    # получаем схожесть между кандитатами
    item_neighbors = neighbors[itemid]
    item_similarities = similarities[itemid]
    
    # получаем рейтинг по пользователю
    uratings = np_ratings[np_ratings[:, 0].astype('int') == userid]
    
    # поиск элементов, которые входят в набор элемента на основе которого рекомендуем
    simitm = uratings[np.isin(uratings[:, 1], item_neighbors)]
    scores = simitm[:, 2]
    indexes = [np.where(item_neighbors == iid)[0][0] for iid in simitm[:,1].astype('int')]    
    sims = item_similarities[indexes]
    
    dot = np.dot(scores, sims)
    som = np.sum(np.abs(sims))

    if dot == 0 or som == 0:
        return mean[userid]
    
    return dot / som

In [107]:
def topn_prediction(userid):
    """
    """
    # топ рекомендаций для выбранного пользователя
    topn = topn_recommendation(userid)
    
    # получаем список элементов
    itemids = topn.movieId.to_list()
    
    predictions = []
    
    # предикт по каждому элементу
    for itemid in itemids:
        r = predict(userid, itemid)
        
        predictions.append((itemid,r))
    
    predictions = pd.DataFrame(predictions, columns=['movieId','prediction'])
    
    # создаем единый и отсортированных набор данных
    topn_predict = pd.merge(topn, predictions, on='movieId', how='inner')
    topn_predict = topn_predict.sort_values(by=['prediction'], ascending=False)
    
    return topn, topn_predict

In [108]:
# получим рекомендации
topn, topn_predict = topn_prediction(userid=2)

In [109]:
topn_predict

,movieId,similarity_with_Iu,title,genres,prediction
0,1,76.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.486842
1,2,76.0,Jumanji (1995),Adventure|Children|Fantasy,3.486842
2,3,76.0,Grumpier Old Men (1995),Comedy|Romance,3.486842
3,5,76.0,Father of the Bride Part II (1995),Comedy,3.486842
4,6,76.0,Heat (1995),Action|Crime|Thriller,3.486842
5,7,76.0,Sabrina (1995),Comedy|Romance,3.486842
6,11,76.0,"American President, The (1995)",Comedy|Drama|Romance,3.486842
7,16,76.0,Casino (1995),Crime|Drama,3.486842
8,19,76.0,Ace Ventura: When Nature Calls (1995),Comedy,3.486842
9,21,76.0,Get Shorty (1995),Comedy|Crime|Thriller,3.486842
